<a href="https://colab.research.google.com/github/felipedram/xf-models-analysis/blob/master/trabalho_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import scipy.optimize as opt

In [0]:
class f(object):
    def __init__(self, solver, goal):
        self.__solver = solver
        if goal == "max" or goal == "min":
            self.__goal = goal
        else:
            raise Exception("goal deve ser 'max' ou 'min'.")
        self.__f_max = None
        self.__x_max = None
        self.__f_min = None
        self.__x_min = None

    @property
    def goal(self):
        return self.__goal

    @property
    def f_max(self):
        return self.__f_max

    @f_max.setter
    def f_max(self, f_max):
        self.__f_max = f_max

    @property
    def x_max(self):
        return self.__x_max

    @x_max.setter
    def x_max(self, x_max):
        self.__x_max = x_max

    @property
    def f_min(self):
        return self.__f_min

    @f_max.setter
    def f_min(self, f_min):
        self.__f_min = f_min

    @property
    def x_min(self):
        return self.__x_min

    @x_min.setter
    def x_min(self, x_min):
        self.__x_min = x_min

    def solve(self, x, inv=False):
        x = np.array(x)
        return self.__solver(x) * -1 if inv else self.__solver(x)

    def mu(self, x):
        x = np.array(x)
        if self.__goal == "max":
            return (self.solve(x) - self.__f_min) / (self.__f_max - self.__f_min)
        elif self.__goal == "min":
            return (self.__f_max - self.solve(x)) / (self.__f_max - self.__f_min)

In [0]:
class f_linear(f):
    def __init__(self, coefs, goal):
        self.__coefs = np.array(coefs)
        f.__init__(self, lambda x: np.dot(self.__coefs, x), goal)

    @property
    def coefs(self):
        return self.__coefs

In [0]:
f1 = f_linear([0.06, 0.53, 0.18, 0.18, 0.06], "max")
f2 = f_linear([25, 70, 60, 95, 45], "max")
f3 = f_linear([0, 32.5, 300, 120, 0], "min")
f4 = f_linear([0.1, 0.1, 0.11, 0.35, 0.33], "min")

In [0]:
A_eq = np.array([[1, 1, 1, 1, 1]])
b_eq = np.array([3000])
x0_bounds = (0, 850)
x1_bounds = (0, 220)
x2_bounds = (0, 1300)
x3_bounds = (0, 1615)
x4_bounds = (0, 700)

In [0]:
F = [f1, f2, f3, f4]
bounds = [x0_bounds, x1_bounds, x2_bounds, x3_bounds, x4_bounds]

In [0]:
from scipy.optimize import linprog
import warnings

for f_ in F:
    res = linprog(f_.coefs * -1, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='simplex')
    if res.success:
        f_.f_max = -1 * res.fun
        f_.x_max = res.x
    else:
        warnings.warn("Otimização mal sucedida.")

In [0]:
for f_ in F:
    res = linprog(f_.coefs, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='simplex')
    if res.success:
        f_.f_min = res.fun
        f_.x_min = res.x
    else:
        warnings.warn("Otimização mal sucedida.")

In [0]:
def mu_D(x, F):
    x = np.array(x)
    return max([f_.mu(x) for f_ in F])

In [0]:
def basinhopping_bounds(**kwargs):
    x = kwargs["x_new"]
    resp = True
    if np.dot(x, A_eq[0]) != b_eq[0]:
        resp = False
    if x[0] < x0_bounds[0] or x[0] > x0_bounds[1]:
        resp = False
    if x[1] < x1_bounds[0] or x[1] > x1_bounds[1]:
        resp = False
    if x[2] < x2_bounds[0] or x[2] > x2_bounds[1]:
        resp = False
    if x[3] < x3_bounds[0] or x[3] > x3_bounds[1]:
        resp = False
    if x[4] < x4_bounds[0] or x[4] > x4_bounds[1]:
        resp = False
    return resp



cobyla_constraints = [
    {"type": "ineq", "fun": lambda x: x[0]},
    {"type": "ineq", "fun": lambda x: x0_bounds[1] - x[0]},
    {"type": "ineq", "fun": lambda x: x[1]},
    {"type": "ineq", "fun": lambda x: x1_bounds[1] - x[1]},
    {"type": "ineq", "fun": lambda x: x[2]},
    {"type": "ineq", "fun": lambda x: x2_bounds[1] - x[2]},
    {"type": "ineq", "fun": lambda x: x[3]},
    {"type": "ineq", "fun": lambda x: x3_bounds[1] - x[3]},
    {"type": "ineq", "fun": lambda x: x[4]},
    {"type": "ineq", "fun": lambda x: x4_bounds[1] - x[4]},
    {"type": "eq", "fun": lambda x: np.dot(x, A_eq[0]) - b_eq[0]},
]

In [0]:
minimizer_kwargs = {"args": F, "method": "SLSQP", "constraints": cobyla_constraints}
res1 = opt.basinhopping(
    mu_D,
    f1.x_max,
    minimizer_kwargs=minimizer_kwargs,
    accept_test=basinhopping_bounds,
    disp=False,
)
res2 = opt.basinhopping(
    mu_D,
    f2.x_max,
    minimizer_kwargs=minimizer_kwargs,
    accept_test=basinhopping_bounds,
    disp=False,
)
res3 = opt.basinhopping(
    mu_D,
    f3.x_max,
    minimizer_kwargs=minimizer_kwargs,
    accept_test=basinhopping_bounds,
    disp=False,
)
res4 = opt.basinhopping(
    mu_D,
    f4.x_max,
    minimizer_kwargs=minimizer_kwargs,
    accept_test=basinhopping_bounds,
    disp=False,
)

In [34]:
print(res1.x)
print(res2.x)
print(res3.x)
print(res4.x)


[5.15183939e+02 3.19403823e-04 9.33099614e+02 9.30816656e+02
 6.20899473e+02]
[4.26481693e+00 2.19077206e+02 1.16260195e+03 1.61384701e+03
 2.09019015e-01]
[2.28927783e+00 8.39487484e+01 1.29874799e+03 1.61392016e+03
 1.09382866e+00]
[2.53148329e+00 1.18538608e+00 6.83137165e+02 1.61314597e+03
 7.00000000e+02]
